In [61]:
import numpy as np 
import os 
import matplotlib.pyplot as plt 
import seaborn as sns
import scipy.io as sp

from tqdm import tqdm
from scipy.linalg import eigh
from numpy.linalg import norm

class Group_dysco_compute: 
    def __init__(self, datafolder, suffix=0): 
        # datafolder (str): absolute path to folder with ONLY subject files
        # suffix to choose load method (int): 0 = txt; 1 = mat
        self.datafolder = datafolder
        self.suffix = suffix

    def get_filename(self):
        files = [f for f in os.listdir(self.datafolder)]

        return files 

    def get_group_eigs(self, n_eigen, half_window_size): 
        # group_eigvec = (neigs x nroi x nf), group_eigval = (neigs x nroi x t x nf)
        group_eigvec = []
        group_eigval = []

        files = self.get_filename()
        if n_eigen > 2 * half_window_size:
            raise ValueError('Number of requested eigenvectors is too large')

        for f in files:      # lucas says this is cubic (o^3) which is very computationally inefficient, find better way to execute 
            file = os.path.join(self.datafolder, f)
            if self.suffix == 1: 
                hold = sp.loadmat(file)
                timeseries = hold['timeSeries']
            else: 
                timeseries = np.genfromtxt(file)

            # calculating eigenvectors and eigenvectors for each subject     
            t, n = timeseries.shape #so here, t is the number of rows, n is the number of columns 
            total_iterations = t - 2 * half_window_size
            progress_bar_eigs = tqdm(total=total_iterations, desc="Calculating eigenvectors and eigenvalues:")
            eigenvectors = np.zeros((t - 2 * half_window_size, n, n_eigen)) #axes of eigenvector array 
            eigenvalues = np.zeros((n_eigen, t - 2 * half_window_size)) #axes of eigenvalue array 

            for i in range(t - 2 * half_window_size):
                truncated_timeseries = timeseries[i:i + 2 * half_window_size, :] #
                z_scored_truncated = (truncated_timeseries - np.mean(truncated_timeseries, axis=0)) / np.std(
                    truncated_timeseries, axis=0, ddof=1)

                normalizing_factor = z_scored_truncated.shape[0] - 1
                z_scored_truncated = (1 / np.sqrt(normalizing_factor)) * z_scored_truncated
                mini_matrix = z_scored_truncated @ z_scored_truncated.T
                ns = len(mini_matrix)

                eigenvalues_t, eigenvectors_t = eigh(mini_matrix, subset_by_index=[ns - n_eigen, ns - 1], overwrite_a=True,
                                                     check_finite=False)
                eigenvectors_t = np.flip(eigenvectors_t, axis=1)
                eigenvalues_t = np.flip(eigenvalues_t, axis=0)
                eigenvalues[:, i] = eigenvalues_t
                eigenvectors[i, :, :] = np.dot(z_scored_truncated.T, eigenvectors_t)

                for j in range(n_eigen):
                    eigenvectors[i, :, j] = eigenvectors[i, :, j] / np.linalg.norm(eigenvectors[i, :, j])
                    eigenvectors[i, :, j] = eigenvectors[i, :, j] * np.sqrt(eigenvalues[j, i])

                progress_bar_eigs.update(1)
                progress_bar_eigs.close()
                
            group_eigvec.append(eigenvectors)
            group_eigval.append(eigenvalues)

        return group_eigvec, group_eigval
    
    def get_group_norm_metastability(self, n_eigen, half_window_size, norm_type=2): 
        group_eigvec, group_eigval = self.get_group_eigs(n_eigen, half_window_size) 
        
        group_norm = []
        group_metastability = []

        for index in range(0, len(group_eigval)):
            data = np.array(group_eigval[index])

            if norm_type == 1:
                norm = np.sum(np.abs(data), axis=0)
            elif norm_type == 2:
                norm = np.sqrt(np.sum(data ** 2, axis=0)) # norm(t)
            elif norm_type == np.inf:
                norm = np.max(data)
                
            metastability =  np.std(norm)  
            group_norm.append(norm)
            group_metastability.append(metastability)

        return group_norm, group_metastability
    
    # add reconfiguration speed and entropy 

    def dysco_distance(self, matrix_a, matrix_b, what_distance): #matrix_a, matrix_b are the eigenvector matrices
        with np.errstate(invalid='ignore'):
            matrix_a = matrix_a.copy()
            matrix_b = matrix_b.copy()

            n_eigen = matrix_a.shape[1]

            # Define minimatrix
            minimatrix = np.zeros((2 * n_eigen, 2 * n_eigen))

            # Fill diagonal with the squared norms of eigenvectors
            for i in range(n_eigen):
                minimatrix[i, i] = np.dot(matrix_a[:, i].T, matrix_a[:, i])
                minimatrix[n_eigen + i, n_eigen + i] = -np.dot(matrix_b[:, i].T, matrix_b[:, i])

            # Fill the rest with scalar products
            minimatrix_up_right = np.dot(matrix_a.T, matrix_b) 
            minimatrix[0:n_eigen, n_eigen:2 * n_eigen] = minimatrix_up_right
            minimatrix[n_eigen:2 * n_eigen, 0:n_eigen] = -minimatrix_up_right.T

            # Compute eigenvalues
            if what_distance != 2:
                lambdas = np.linalg.eigvals(minimatrix)
                lambdas = np.real(lambdas)

            if what_distance == 1:
                distance = np.sum(np.abs(lambdas))
            elif what_distance == 2:
                distance = np.sqrt(np.sum(np.diag(minimatrix) ** 2) - 2 * np.sum(minimatrix_up_right ** 2))
            else:
                distance = np.max(lambdas) #default 

        return distance

    def compute_fcd(self, eigenvectors, dysco_distance, what_distance=0):
        # Get the number of time points
        T = eigenvectors.shape[0]
        
        # Initialize the FCD matrix
        FCD = np.zeros((T, T))
        
        # Loop over all time points to compute the FCD matrix
        for i in range(T):
            for j in range(i + 1, T):
                # Compute the Dysco distance between the eigenvectors at time points i and j
                FCD[i, j] = dysco_distance(eigenvectors[i, :, :], eigenvectors[j, :, :], what_distance)
                # Ensure the matrix is symmetric
                FCD[j, i] = FCD[i, j]
        
        return FCD
    
    def get_group_reconfigurationspeed(self, what_distance=0, lag=10, n_eigen=10, half_window_size=20): # but if we've already calculated eigvec, eigval for norm_metastability we shouldn't need to do it again 
        group_eigvec, group_eigval = self.get_group_eigs(n_eigen, half_window_size)
        group_speed = [] 

        for index in range(0, len(group_eigvec)): 
            eigenvectors = np.array(group_eigvec[index])
            FCD = self.compute_fcd(eigenvectors, self.dysco_distance, what_distance)
            print(FCD.shape)
        # Determine the number of time points
            T = FCD.shape[0]
            # Initialize the speed array
            speed = np.zeros(T - lag)
            # Calculate the reconfiguration speed
            for i in range(T - lag):
                speed[i] = FCD[i, i + lag]
            
            print(speed.shape)
            group_speed.append(speed)

        return group_speed 
    
    def dysco_entropy(eigenvalues):
        group_eigvec, group_eigval = self.get_group_eigs(n_eigen, half_window_size)
        group_von_neumann = [] 

        for index in range(0, len(group_eigval)): 
            eigenvalues = np.array(group_eigval[index])
            # Ensure eigenvalues are a numpy array
            eigenvalues = np.array(eigenvalues)
            # Number of eigenvalues
            n_eigenvalues = eigenvalues.shape[0]
            # Normalize the eigenvalues using np.tile
            von_neumann = eigenvalues / np.tile(np.sum(eigenvalues, axis=0), (n_eigenvalues, 1))
            # Avoid log(0) by setting zero values to a small positive number
            von_neumann[von_neumann == 0] = np.finfo(float).eps
            # Calculate von Neumann entropy
            von_neumann = -np.sum(np.log(von_neumann) * von_neumann, axis=0)
            group_von_neumann.append(von_neumann)
        
        return group_von_neumann

In [62]:
class Group_dysco_analysis: 
    ##
    ##
    ##
    ##
    ##
    ##
    ##
    ##

SyntaxError: incomplete input (3579702744.py, line 6)

In [ ]:
ketamine = Group_dysco_compute('/Users/emiliewielezynski/Desktop/ketamine_paper/humans/parcellated_raw/keta/', 1)


In [56]:
group_speed = ketamine.get_group_reconfigurationspeed()

Calculating eigenvectors and eigenvalues::   0%|          | 1/410 [00:00<00:03, 104.12it/s]


(410, 410)
(400,)
(410, 410)
(400,)
(410, 410)
(400,)
(410, 410)
(400,)
(410, 410)
(400,)
(410, 410)
(400,)
(410, 410)
(400,)
(410, 410)
(400,)
(410, 410)
(400,)
(410, 410)
(400,)
(410, 410)
(400,)
(410, 410)
(400,)
(410, 410)
(400,)
(410, 410)
(400,)
(410, 410)
(400,)
(410, 410)
(400,)


In [37]:
# group_speed.shape = (16, 400) (timepoints - lag)
group_eigvec, group_eigval = ketamine.get_group_eigs(10, 20)

Calculating eigenvectors and eigenvalues::   0%|          | 1/410 [00:00<00:02, 138.99it/s]
